In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
len(dataset)

In [ ]:
plt.scatter(dataset["bmi"], dataset["expenses"])
plt.xlabel("bmi")
plt.ylabel("expenses")
plt.show()

In [ ]:
plt.scatter(dataset["sex"],dataset["expenses"])
plt.show()

In [ ]:
plt.scatter(dataset["smoker"],dataset["expenses"])
plt.show()

In [ ]:
plt.scatter(dataset["children"],dataset["expenses"])
plt.show()

In [ ]:
plt.scatter(dataset["region"],dataset["expenses"])
plt.show()

In [ ]:
train_dataset = dataset.sample(frac = 0.8)
test_dataset = dataset.drop(train_dataset.index)


In [ ]:
print(len(train_dataset))

In [ ]:
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")
train_labels.head()

In [ ]:
plt.scatter(train_dataset["sex"],train_labels)
plt.show()

In [ ]:
train_dataset.head()

In [ ]:
categorical_columns = ["sex","smoker","region"]
numerical_columns = ["age","bmi","children"]

features_columns_encoding = []
for encoding_features in categorical_columns:
  encoding = train_dataset[encoding_features].unique()
  features_columns_encoding.append(encoding)
print(features_columns_encoding)

In [ ]:
def encoding_features(dataset, key):
    indice_encoding = float(0)
    features_encoding = []
    dict_features_encoding = []
    features_encoding.append(dataset[key].unique())
    for i in range(len(features_encoding[0])):
        dataset.loc[dataset[key] == features_encoding[0][i], key] = indice_encoding
        dict_features_encoding.append({features_encoding[0][i]: indice_encoding})
        indice_encoding += float(1)
    return dataset


In [ ]:
encoding_features(test_dataset,"sex")
encoding_features(test_dataset,"smoker")
encoding_features(test_dataset,"region")
test_dataset.head()

In [ ]:
features_test = list()
list_key = ["sex","smoker","region"]
for i in list_key:
  features_test.append(test_dataset[i].unique())
print(features_test)

In [ ]:
encoding_features(train_dataset,"sex")
encoding_features(train_dataset,"smoker")
encoding_features(train_dataset,"region")
train_dataset.head()

In [ ]:
train_dataset = np.array(train_dataset, dtype = np.float64)
test_dataset = np.array(test_dataset,dtype = np.float64)

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
normalizer = layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))# dtype float 32 is needed because without this the array cannot be put into a tensor

model = keras.Sequential([
    normalizer,
    layers.Dense(16,kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(4,kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.2),
    layers.Dense(1),
])

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.01), loss = "mae", metrics = ["mae","mse"])
model.build()
model.summary()

In [ ]:
model.fit(train_dataset,train_labels,epochs = 100, validation_split = 0.5)

In [ ]:
#RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
